In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup, Comment
import pandas as pd

In [2]:
import pymongo
import pandas as pd
import numpy as np

client = pymongo.MongoClient("localhost", 27017)

db = client['nba_teste4']

### List all games of specific season

In [3]:
def get_months(season):
    year = str(season)
    url = "https://www.basketball-reference.com/leagues/NBA_" + year + "_games.html"
    html = urlopen(url)
    soup = BeautifulSoup(html, 'html.parser')

    links = [a['href'] for a in soup.find_all('a', href=True)]
    link_months = [link for link in links if ('NBA_'+year+'_games-' in link) and ('html' in link)]
    return link_months

def get_list_of_games(urlarg):
    url = "https://www.basketball-reference.com"+urlarg
    html = urlopen(url)
    soup = BeautifulSoup(html, 'html.parser')

    links = [a['href'] for a in soup.find_all('a', href=True)]
    link_bs = [link for link in links if ('boxscore' in link) and ('html' in link)]
    return link_bs

In [4]:
season = 2020
link_months = get_months(season)

games = [] #list for the url for all games of the season
for i in link_months:
    games = games + get_list_of_games(i)
games

['/boxscores/201910220TOR.html',
 '/boxscores/201910220LAC.html',
 '/boxscores/201910230CHO.html',
 '/boxscores/201910230IND.html',
 '/boxscores/201910230ORL.html',
 '/boxscores/201910230BRK.html',
 '/boxscores/201910230MIA.html',
 '/boxscores/201910230PHI.html',
 '/boxscores/201910230DAL.html',
 '/boxscores/201910230SAS.html',
 '/boxscores/201910230UTA.html',
 '/boxscores/201910230PHO.html',
 '/boxscores/201910230POR.html',
 '/boxscores/201910240DET.html',
 '/boxscores/201910240HOU.html',
 '/boxscores/201910240GSW.html',
 '/boxscores/201910250BOS.html',
 '/boxscores/201910250CHO.html',
 '/boxscores/201910250BRK.html',
 '/boxscores/201910250MEM.html',
 '/boxscores/201910250NOP.html',
 '/boxscores/201910250OKC.html',
 '/boxscores/201910250DEN.html',
 '/boxscores/201910250SAC.html',
 '/boxscores/201910250LAL.html',
 '/boxscores/201910260MIL.html',
 '/boxscores/201910260DET.html',
 '/boxscores/201910260ATL.html',
 '/boxscores/201910260NYK.html',
 '/boxscores/201910260CHI.html',
 '/boxscor

### Random games urls

In [4]:
url = "https://www.basketball-reference.com/boxscores/201810160BOS.html"
# this is the HTML from the given URL
html = urlopen(url)
soup1 = BeautifulSoup(html)

### Functions to get match_scores and player's box scores

In [3]:
def player_avg_stat(player, stat):
    player = str(player)
    stat = str(stat)
    cursor = db.games.find({"teamA_stats."+player:{"$exists":True}},{"teamA_stats."+player+"."+stat:1,"_id":False})
    value = [doc['teamA_stats'][player][stat] for doc in cursor]
    cursor1 = db.games.find({"teamH_stats."+player:{"$exists":True}},{"teamH_stats."+player+"."+stat:1,"_id":False})
    value1 = [doc['teamH_stats'][player][stat] for doc in cursor1]
    values = value + value1
    num_values = [float(num) for num in values if num is not None]
    stat_mean = np.array(num_values).mean()
    return player, stat, stat_mean

In [4]:
def get_players_score(team):
    id_str = "box-"+str(team)+"-game-basic"
    soup2 = soup.find_all(id=id_str)
    player_stats = [[td.getText() for td in soup2[0].find_all('tr')[i:][0].find_all('td')]
                for i in range(2,len(soup2[0].find_all('tr'))-1)]
    player_names = [[td.getText() for td in soup2[i].find_all('a')]
                for i in range(len(soup2))]
    player_names = [name.replace('.','') for name in player_names[0]]
    player_stats = [player for player in player_stats if len(player) > 0]
    headers = [th.getText() for th in soup.findAll('tr')[1].findAll('th')]
    box_score = pd.DataFrame(player_stats, columns=headers[1:], index = player_names)
    
    return box_score

In [5]:
def get_game_score(soup):
    
    for tr in soup.find_all(id='all_line_score'):
        comment = tr.find(text=lambda text:isinstance(text, Comment))
        commentsoup = BeautifulSoup(comment , 'lxml')
        
    rows = commentsoup.findAll('tr')
    game_score = [[td.getText() for td in rows[i].findAll('td')]
                for i in range(len(rows))]
    game_score = [score for score in game_score if len(score) > 0]

    header = [[td.getText() for td in rows[i].findAll('th')]
                for i in range(1,len(rows))]
    header = [head for head in header if len(head) > 0]
    header = header[0]
    teamA = game_score[0][0]
    teamH = game_score[1][0]
    
    return pd.DataFrame(game_score, columns=header).set_index('\xa0'), teamA, teamH

In [6]:
games_dict_list = []
for i in range(3):
    url = "https://www.basketball-reference.com" + games[i]
    html = urlopen(url)
    soup = BeautifulSoup(html)
    game_df, teamA, teamH = get_game_score(soup)
    teamA_stats = get_players_score(teamA)
    teamH_stats = get_players_score(teamH)
    game_id = url[-17:-5]
    game_date = url[-17:-9]
    
    game_dict = {'game_id':game_id, 'game_date':game_date, 'teamA':teamA,
                 'teamH':teamH, 'game_score':game_df.to_dict(orient='index'),
                'teamA_stats':teamA_stats.to_dict(orient='index'),
                'teamH_stats':teamH_stats.to_dict(orient='index')}
    games_dict_list.append(game_dict)
    print(game_df)
    print(teamA_stats)
    print(teamH_stats)

NameError: name 'games' is not defined

### Get names of all players

In [78]:
def list_of_names():
    cursor2 = db.games.find({},{"teamA":1,"teamA_stats":1,"_id":False})
    c = pd.DataFrame(cursor2)
    total_players = pd.DataFrame()
    for j in range(len(c)):
        players = c.to_dict(orient='split')['data'][j][1]
        team = c.to_dict(orient='split')['data'][j][0]
        name_players = [name for name in pd.DataFrame(players).columns]
        team_list = [team for i in range(len(pd.DataFrame(players).columns))]
        total_players = total_players.append(pd.DataFrame({'teams':team_list, 'players':name_players}),ignore_index=True)
        total_players1 = total_players.drop_duplicates(subset ="players", keep = 'first', inplace = False)
    return total_players1

In [ ]:
nomes = list_of_names()
nomes

### Data Frame of avg stats

In [32]:
def player_avg_stat(player, stat):
    player = str(player)
    stat = str(stat)
    cursor = db.games.find({"teamA_stats."+player:{"$exists":True}},{"teamA_stats."+player+"."+stat:1,"_id":False})
    value = [doc['teamA_stats'][player][stat] for doc in cursor]
    cursor1 = db.games.find({"teamH_stats."+player:{"$exists":True}},{"teamH_stats."+player+"."+stat:1,"_id":False})
    value1 = [doc['teamH_stats'][player][stat] for doc in cursor1]
    values = value + value1
    num_values = [float(num) for num in values if (num is not None) and (num is not '')]
    stat_mean = np.array(num_values).mean()
    return player, stat, stat_mean

Build dataframe

In [33]:
def get_avgs_dataframe():
    stats = ['FG','FGA','FG%','3P','3PA','3P%','FT','FTA','FT%','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS','+/-']
    names = list(total_players['players'])
    store = []
    final_list = []
    avg_stats = pd.DataFrame()
    for name in names:
        store = [name]
        for stat in stats:
            foo = player_avg_stat(name, stat)
            store = store + [foo[2]]
        final_list.append(store)
    return pd.DataFrame(final_list, columns=(['Players'] +stats)).sort_values('TRB', ascending=False)

In [38]:
oi = get_avgs_dataframe()

/home/pedro/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: Mean of empty slice.
  # Remove the CWD from sys.path while we load stuff.
/home/pedro/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [39]:
oi

,Players,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-
3,Nikola Jokić,6.000000,12.500000,0.519167,1.000000,3.500000,0.171500,2.833333,3.500000,0.838000,2.000000,8.500000,10.500000,6.000000,1.166667,0.500000,3.000000,3.333333,15.833333,5.333333
2,Will Barton,4.250000,11.000000,0.349750,1.500000,3.500000,0.458500,4.000000,4.750000,0.771000,2.250000,4.000000,6.250000,1.500000,1.250000,1.000000,1.250000,1.250000,14.000000,8.250000
4,Paul Millsap,3.833333,8.833333,0.403833,1.166667,2.833333,0.264200,2.833333,3.000000,0.916667,1.333333,4.000000,5.333333,0.666667,0.666667,0.666667,0.833333,2.833333,11.666667,4.000000
7,Mason Plumlee,2.166667,3.666667,0.466667,0.000000,0.166667,0.000000,1.833333,2.833333,0.683400,2.166667,2.833333,5.000000,2.666667,0.500000,0.500000,1.000000,3.000000,6.166667,-5.333333
0,Jamal Murray,6.333333,14.833333,0.428333,1.666667,4.833333,0.350500,4.166667,4.666667,0.928000,0.666667,4.166667,4.833333,3.500000,0.833333,0.166667,2.000000,2.500000,18.500000,4.833333
8,Torrey Craig,1.166667,4.333333,0.340333,0.333333,2.166667,0.133400,0.333333,0.666667,0.500000,1.000000,3.333333,4.333333,0.833333,0.500000,1.333333,0.500000,2.333333,3.000000,-5.000000
11,Michael Porter,3.000000,5.000000,0.562500,0.500000,1.500000,0.333000,2.500000,4.000000,0.583500,1.000000,3.000000,4.000000,0.500000,0.000000,0.000000,1.000000,1.000000,9.000000,0.500000
1,Gary Harris,4.500000,10.666667,0.428833,1.833333,5.000000,0.383333,1.833333,2.333333,0.825000,0.333333,3.166667,3.500000,1.666667,1.333333,0.166667,1.666667,2.166667,12.666667,2.166667
5,Jerami Grant,3.000000,7.166667,0.396167,0.666667,3.166667,0.233333,1.333333,2.166667,0.604250,1.000000,2.166667,3.166667,0.833333,0.333333,0.333333,1.333333,2.000000,8.000000,-3.833333
6,Malik Beasley,3.166667,8.666667,0.351667,1.833333,4.333333,0.413833,0.500000,0.500000,1.000000,0.333333,1.166667,1.500000,0.833333,1.000000,0.166667,1.333333,1.833333,8.666667,-3.666667


In [12]:
pd.DataFrame(final_list, columns=(['Players'] +stats)).sort_values('TRB', ascending=False)

,Players,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-
39,Andre Drummond,8.750000,15.625000,0.565875,0.000000,0.750000,0.000000,3.750000,5.625000,0.691625,5.000000,14.375000,19.375000,2.625000,1.750000,2.500000,4.000000,4.250000,21.250000,-4.500000
52,Kevin Love,6.142857,12.428571,0.503429,2.571429,6.571429,0.334714,4.000000,4.571429,0.884143,0.857143,13.571429,14.428571,4.142857,1.000000,0.428571,3.285714,1.714286,18.857143,-1.285714
183,Giannis Antetokounmpo,10.571429,17.285714,0.604000,0.571429,3.285714,0.164286,6.000000,10.000000,0.596143,1.857143,12.142857,14.000000,7.428571,1.142857,1.571429,4.142857,4.142857,27.714286,10.000000
251,Hassan Whiteside,5.833333,8.833333,0.643167,0.000000,0.000000,NaN,2.666667,3.833333,0.746800,3.333333,8.666667,12.000000,0.666667,0.500000,1.333333,2.333333,3.500000,14.333333,0.333333
260,Rudy Gobert,3.714286,6.000000,0.649571,0.000000,0.000000,NaN,3.428571,5.285714,0.501000,1.857143,10.142857,12.000000,0.857143,1.000000,1.142857,2.857143,3.285714,10.857143,5.571429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,Pau Gasol,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
414,Naz Mitrou-Long,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
415,Brian Bowen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
416,Rajon Rondo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [147]:
pd.DataFrame({'teams':team_list, 'players':name_players})

,teams,players
0,DET,Andre Drummond
1,DET,Tony Snell
2,DET,Bruce Brown
3,DET,Markieff Morris
4,DET,Reggie Jackson
5,DET,Luke Kennard
6,DET,Derrick Rose
7,DET,Langston Galloway
8,DET,Thon Maker
9,DET,Christian Wood


### All scores df

In [50]:
cursor3 = db.games.find({},{"game_score":1, "_id":False}).limit(1)

In [32]:
pd.DataFrame(cursor3)

,game_id,game_score
0,201910220TOR,"{'NOP': {'1': '30', '2': '31', '3': '25', '4':..."


In [60]:
pd.DataFrame.from_dict(cursor3[0]['game_score'], orient='index')


,1,2,3,4,OT,T
NOP,30,31,25,31,5,122
TOR,27,29,32,29,13,130


In [ ]:
['game_score'])[0]

In [19]:
games_dict_list

[{'game_id': '201910220TOR',
  'game_date': '20191022',
  'teamA': 'NOP',
  'teamH': 'TOR',
  'game_score': {'NOP': {'1': '30',
    '2': '31',
    '3': '25',
    '4': '31',
    'OT': '5',
    'T': '122'},
   'TOR': {'1': '27',
    '2': '29',
    '3': '32',
    '4': '29',
    'OT': '13',
    'T': '130'}},
  'teamA_stats': {'Jrue Holiday': {'MP': '41:05',
    'FG': '6',
    'FGA': '15',
    'FG%': '.400',
    '3P': '1',
    '3PA': '6',
    '3P%': '.167',
    'FT': '0',
    'FTA': '2',
    'FT%': '.000',
    'ORB': '2',
    'DRB': '2',
    'TRB': '4',
    'AST': '6',
    'STL': '0',
    'BLK': '2',
    'TOV': '5',
    'PF': '2',
    'PTS': '13',
    '+/-': '-14'},
   'Brandon Ingram': {'MP': '35:06',
    'FG': '8',
    'FGA': '19',
    'FG%': '.421',
    '3P': '2',
    '3PA': '5',
    '3P%': '.400',
    'FT': '4',
    'FTA': '4',
    'FT%': '1.000',
    'ORB': '0',
    'DRB': '5',
    'TRB': '5',
    'AST': '5',
    'STL': '1',
    'BLK': '2',
    'TOV': '2',
    'PF': '4',
    'PTS': '22

In [ ]:
# ‘dict’, ‘list’, ‘series’, ‘split’, ‘records’, ‘index’)

In [20]:
game_df.to_dict(orient='index')

{'CHI': {'1': '28', '2': '27', '3': '40', '4': '30', 'T': '125'},
 'CHO': {'1': '37', '2': '26', '3': '33', '4': '30', 'T': '126'}}

In [21]:
teamA_stats.to_dict(orient='index')

{'Lauri Markkanen': {'MP': '34:22',
  'FG': '13',
  'FGA': '25',
  'FG%': '.520',
  '3P': '1',
  '3PA': '7',
  '3P%': '.143',
  'FT': '8',
  'FTA': '10',
  'FT%': '.800',
  'ORB': '5',
  'DRB': '12',
  'TRB': '17',
  'AST': '2',
  'STL': '1',
  'BLK': '0',
  'TOV': '1',
  'PF': '1',
  'PTS': '35',
  '+/-': '-8'},
 'Zach LaVine': {'MP': '30:00',
  'FG': '7',
  'FGA': '17',
  'FG%': '.412',
  '3P': '1',
  '3PA': '4',
  '3P%': '.250',
  'FT': '1',
  'FTA': '1',
  'FT%': '1.000',
  'ORB': '3',
  'DRB': '3',
  'TRB': '6',
  'AST': '7',
  'STL': '2',
  'BLK': '1',
  'TOV': '5',
  'PF': '3',
  'PTS': '16',
  '+/-': '-8'},
 'Wendell Carter': {'MP': '29:25',
  'FG': '5',
  'FGA': '11',
  'FG%': '.455',
  '3P': '0',
  '3PA': '0',
  '3P%': '',
  'FT': '2',
  'FTA': '2',
  'FT%': '1.000',
  'ORB': '5',
  'DRB': '4',
  'TRB': '9',
  'AST': '0',
  'STL': '0',
  'BLK': '1',
  'TOV': '1',
  'PF': '4',
  'PTS': '12',
  '+/-': '+2'},
 'Otto Porter': {'MP': '27:43',
  'FG': '3',
  'FGA': '10',
  'FG%': '

In [22]:
games_dict_list = []
for i in range(3):
    url = "https://www.basketball-reference.com" + games[i]
    html = urlopen(url)
    soup = BeautifulSoup(html)
    game_df, teamA, teamH = get_game_score(soup)
    teamA_stats = get_players_score(teamA)
    teamH_stats = get_players_score(teamH)
    game_id = url[-17:-5]
    game_date = url[-17:-9]
    
    game_dict = {'game_id':game_id, 'game_date':game_date, 'teamA':teamA,
                 'teamH':teamH, 'game_score':game_df.to_dict(orient='index')
                }
    games_dict_list.append(game_dict)
    print(game_df)
    print(teamA_stats)
    print(teamH_stats)

      1   2   3   4  OT    T
                            
NOP  30  31  25  31   5  122
TOR  27  29  32  29  13  130
                                    MP    FG   FGA    FG%    3P   3PA   3P%  \
Jrue Holiday                     41:05     6    15   .400     1     6  .167   
Brandon Ingram                   35:06     8    19   .421     2     5  .400   
JJ Redick                        27:03     6     9   .667     4     6  .667   
Lonzo Ball                       24:50     2     7   .286     2     3  .667   
Derrick Favors                   20:46     3     6   .500     0     0         
Josh Hart                        28:10     4     9   .444     3     5  .600   
Nicolò Melli                     19:37     5     7   .714     4     5  .800   
Kenrich Williams                 18:02     0     4   .000     0     2  .000   
Frank Jackson                    13:51     3     6   .500     1     3  .333   
Jahlil Okafor                    12:29     3     3  1.000     0     0         
E'Twaun Moore  

      1   2   3   4    T
                        
CHI  28  27  40  30  125
CHO  37  26  33  30  126
                              MP    FG   FGA   FG%    3P   3PA    3P%    FT  \
Lauri Markkanen            34:22    13    25  .520     1     7   .143     8   
Zach LaVine                30:00     7    17  .412     1     4   .250     1   
Wendell Carter             29:25     5    11  .455     0     0            2   
Otto Porter                27:43     3    10  .300     1     5   .200     2   
Tomáš Satoranský           24:22     1     5  .200     0     0            0   
Coby White                 27:09     6    13  .462     1     4   .250     4   
Thaddeus Young             25:24     7    11  .636     3     5   .600     0   
Kris Dunn                  23:50     5     7  .714     0     2   .000     1   
Ryan Arcidiacono           10:56     2     3  .667     2     2  1.000     0   
Luke Kornet                 6:49     0     3  .000     0     1   .000     0   
Denzel Valentine    Did Not Pla

In [19]:
url = "https://www.basketball-reference.com/boxscores/201810160BOS.html"

In [22]:
game_id = url[-17:-5]

'201810160BOS'

In [37]:
game_df.to_dict(orient='index')

{'CHI': {'1': '28', '2': '27', '3': '40', '4': '30', 'T': '125'},
 'CHO': {'1': '37', '2': '26', '3': '33', '4': '30', 'T': '126'}}

In [36]:
teamH_stats.to_dict(orient='index')

{('PJ Washington',): {'MP': '37:37',
  'FG': '9',
  'FGA': '17',
  'FG%': '.529',
  '3P': '7',
  '3PA': '11',
  '3P%': '.636',
  'FT': '2',
  'FTA': '2',
  'FT%': '1.000',
  'ORB': '1',
  'DRB': '3',
  'TRB': '4',
  'AST': '1',
  'STL': '1',
  'BLK': '1',
  'TOV': '4',
  'PF': '2',
  'PTS': '27',
  '+/-': '+8'},
 ('Cody Zeller',): {'MP': '35:06',
  'FG': '6',
  'FGA': '11',
  'FG%': '.545',
  '3P': '0',
  '3PA': '2',
  '3P%': '.000',
  'FT': '3',
  'FTA': '4',
  'FT%': '.750',
  'ORB': '4',
  'DRB': '8',
  'TRB': '12',
  'AST': '1',
  'STL': '0',
  'BLK': '0',
  'TOV': '1',
  'PF': '4',
  'PTS': '15',
  '+/-': '+3'},
 ('Dwayne Bacon',): {'MP': '33:06',
  'FG': '8',
  'FGA': '21',
  'FG%': '.381',
  '3P': '2',
  '3PA': '8',
  '3P%': '.250',
  'FT': '4',
  'FTA': '6',
  'FT%': '.667',
  'ORB': '1',
  'DRB': '4',
  'TRB': '5',
  'AST': '3',
  'STL': '2',
  'BLK': '0',
  'TOV': '2',
  'PF': '2',
  'PTS': '22',
  '+/-': '-3'},
 ('Miles Bridges',): {'MP': '31:42',
  'FG': '4',
  'FGA': '7',


In [40]:
teamH_stats.reset_index

<bound method DataFrame.reset_index of                                   MP    FG   FGA   FG%    3P   3PA    3P%  \
PJ Washington                  37:37     9    17  .529     7    11   .636   
Cody Zeller                    35:06     6    11  .545     0     2   .000   
Dwayne Bacon                   33:06     8    21  .381     2     8   .250   
Miles Bridges                  31:42     4     7  .571     0     2   .000   
Terry Rozier                   30:24     2    10  .200     2     6   .333   
Devonte' Graham                27:19     7     9  .778     6     7   .857   
Marvin Williams                16:53     6     9  .667     5     7   .714   
Malik Monk                     16:41     3     4  .750     1     1  1.000   
Nicolas Batum                  11:12     0     0           0     0          
Willy Hernangómez       Did Not Play  None  None  None  None  None   None   
Michael Kidd-Gilchrist  Did Not Play  None  None  None  None  None   None   
Caleb Martin            Did Not Play 

In [42]:
teamH_stats.index

MultiIndex([(         'PJ Washington',),
            (           'Cody Zeller',),
            (          'Dwayne Bacon',),
            (         'Miles Bridges',),
            (          'Terry Rozier',),
            (       'Devonte' Graham',),
            (       'Marvin Williams',),
            (            'Malik Monk',),
            (         'Nicolas Batum',),
            (     'Willy Hernangómez',),
            ('Michael Kidd-Gilchrist',),
            (          'Caleb Martin',),
            (       'Bismack Biyombo',)],
           )

In [44]:
url = "https://www.basketball-reference.com/boxscores/201911010BRK.html"
# this is the HTML from the given URL
html = urlopen(url)
soup = BeautifulSoup(html)

In [68]:
    id_str = "box-BRK-game-basic"
    soup2 = soup.find_all(id=id_str)
    player_stats = [[td.getText() for td in soup2[0].find_all('tr')[i:][0].find_all('td')]
                for i in range(2,len(soup2[0].find_all('tr'))-1)]
    player_names = [[td.getText() for td in soup2[i].find_all('a')]
                for i in range(len(soup2))]
    player_names = [name.replace('.','') for name in player_names[0]]

In [71]:
player_names

['Kyrie Irving',
 'Taurean Waller-Prince',
 'Joe Harris',
 'Caris LeVert',
 'Jarrett Allen',
 'Garrett Temple',
 'Spencer Dinwiddie',
 'DeAndre Jordan',
 'Rodions Kurucs',
 'Džanan Musa',
 'Theo Pinson',
 'David Nwaba',
 'Wilson Chandler']

In [73]:
    player_stats = [player for player in player_stats if len(player) > 0]
    headers = [th.getText() for th in soup1.findAll('tr')[1].findAll('th')]
    box_score = pd.DataFrame(player_stats, columns=headers[1:], index = player_names)

In [55]:
player_names[0]

['Kyrie Irving',
 'Taurean Waller-Prince',
 'Joe Harris',
 'Caris LeVert',
 'Jarrett Allen',
 'Garrett Temple',
 'Spencer Dinwiddie',
 'DeAndre Jordan',
 'Rodions Kurucs',
 'Džanan Musa',
 'Theo Pinson',
 'David Nwaba',
 'Wilson Chandler']

In [65]:
new_index = [name.replace('.','') for name in box_score.index]

In [66]:
new_index

['Kyrie Irving',
 'Turen Wller-Prince',
 'Joe Hrris',
 'Cris LeVert',
 'Jrrett Allen',
 'Grrett Temple',
 'Spencer Dinwiddie',
 'DeAndre Jordn',
 'Rodions Kurucs',
 'Džnn Mus',
 'Theo Pinson',
 'Dvid Nwb',
 'Wilson Chndler']

In [75]:
box_score

,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-
Kyrie Irving,34:29,7,18,.389,5,8,.625,3,4,.750,0,4,4,10,0,1,6,3,22,+10
Taurean Waller-Prince,34:25,9,17,.529,6,10,.600,3,4,.750,1,11,12,1,1,1,1,3,27,+18
Joe Harris,34:01,5,9,.556,3,4,.750,2,2,1.000,2,7,9,2,3,0,1,2,15,+12
Caris LeVert,32:13,8,15,.533,1,2,.500,8,9,.889,1,3,4,4,1,0,5,4,25,+18
Jarrett Allen,31:02,3,3,1.000,0,0,,1,4,.250,2,8,10,1,0,2,2,1,7,+29
Garrett Temple,24:03,6,9,.667,4,6,.667,0,0,,1,6,7,3,2,2,0,0,16,+4
Spencer Dinwiddie,19:48,2,7,.286,0,2,.000,1,2,.500,0,1,1,2,1,1,2,4,5,-14
DeAndre Jordan,16:17,2,5,.400,0,0,,2,3,.667,2,5,7,3,0,0,0,2,6,-19
Rodions Kurucs,9:17,0,2,.000,0,0,,0,0,,1,0,1,0,0,0,1,2,0,-6
Džanan Musa,4:25,0,0,,0,0,,0,0,,0,0,0,0,0,0,2,0,0,-17


In [68]:
cursor = db.games.find({},{"game_id":1,"_id":False})
old_games = ['/boxscores/' + doc['game_id'] + '.html' for doc in cursor]

In [69]:
old_games

['/boxscores/201910220TOR.html',
 '/boxscores/201910220LAC.html',
 '/boxscores/201910230CHO.html',
 '/boxscores/201910230IND.html',
 '/boxscores/201910230ORL.html',
 '/boxscores/201910230BRK.html',
 '/boxscores/201910230MIA.html',
 '/boxscores/201910230PHI.html',
 '/boxscores/201910230DAL.html',
 '/boxscores/201910230SAS.html',
 '/boxscores/201910230UTA.html',
 '/boxscores/201910230PHO.html',
 '/boxscores/201910230POR.html',
 '/boxscores/201910240DET.html',
 '/boxscores/201910240HOU.html',
 '/boxscores/201910240GSW.html',
 '/boxscores/201910250BOS.html',
 '/boxscores/201910250CHO.html',
 '/boxscores/201910250BRK.html',
 '/boxscores/201910250MEM.html',
 '/boxscores/201910250NOP.html',
 '/boxscores/201910250OKC.html',
 '/boxscores/201910250DEN.html',
 '/boxscores/201910250SAC.html',
 '/boxscores/201910250LAL.html',
 '/boxscores/201910260MIL.html',
 '/boxscores/201910260DET.html',
 '/boxscores/201910260ATL.html',
 '/boxscores/201910260NYK.html',
 '/boxscores/201910260CHI.html',
 '/boxscor

In [62]:
games

['/boxscores/201910220TOR.html',
 '/boxscores/201910220LAC.html',
 '/boxscores/201910230CHO.html',
 '/boxscores/201910230IND.html',
 '/boxscores/201910230ORL.html',
 '/boxscores/201910230BRK.html',
 '/boxscores/201910230MIA.html',
 '/boxscores/201910230PHI.html',
 '/boxscores/201910230DAL.html',
 '/boxscores/201910230SAS.html',
 '/boxscores/201910230UTA.html',
 '/boxscores/201910230PHO.html',
 '/boxscores/201910230POR.html',
 '/boxscores/201910240DET.html',
 '/boxscores/201910240HOU.html',
 '/boxscores/201910240GSW.html',
 '/boxscores/201910250BOS.html',
 '/boxscores/201910250CHO.html',
 '/boxscores/201910250BRK.html',
 '/boxscores/201910250MEM.html',
 '/boxscores/201910250NOP.html',
 '/boxscores/201910250OKC.html',
 '/boxscores/201910250DEN.html',
 '/boxscores/201910250SAC.html',
 '/boxscores/201910250LAL.html',
 '/boxscores/201910260MIL.html',
 '/boxscores/201910260DET.html',
 '/boxscores/201910260ATL.html',
 '/boxscores/201910260NYK.html',
 '/boxscores/201910260CHI.html',
 '/boxscor

In [64]:
new_games = [game for game in games if game not in list]

In [70]:
len(new_games), len(games), len(old_games)

(114, 214, 100)

In [168]:
total_players.drop_duplicates(subset ="players", keep = 'first', inplace = True)

In [117]:
players = c.to_dict(orient='split')['data'][3][1]
team = c.to_dict(orient='split')['data'][3][0]
name_players = [name for name in pd.DataFrame(players).columns]
team_list = [team for i in range(len(pd.DataFrame(players).columns))]
total_players = pd.DataFrame({'teams':team_list, 'players':name_players})

total_players.drop_duplicates(subset ="players", keep = 'first', inplace = True)

In [140]:
name_players = [name for name in pd.DataFrame(players).columns]

In [135]:
len(pd.DataFrame(players).columns)

13

In [283]:
list(total_players['players'])

['Jrue Holiday',
 'Brandon Ingram',
 'JJ Redick',
 'Lonzo Ball',
 'Derrick Favors',
 'Josh Hart',
 'Nicolò Melli',
 'Kenrich Williams',
 'Frank Jackson',
 'Jahlil Okafor',
 "E'Twaun Moore",
 'Nickeil Alexander-Walker',
 'Jaxson Hayes',
 'Anthony Davis',
 'LeBron James',
 'Danny Green',
 'Avery Bradley',
 'JaVale McGee',
 'Kentavious Caldwell-Pope',
 'Dwight Howard',
 'Quinn Cook',
 'Troy Daniels',
 'Jared Dudley',
 'Alex Caruso',
 'Zach Norvell',
 'Kostas Antetokounmpo',
 'Lauri Markkanen',
 'Zach LaVine',
 'Wendell Carter',
 'Otto Porter',
 'Tomáš Satoranský',
 'Coby White',
 'Thaddeus Young',
 'Kris Dunn',
 'Ryan Arcidiacono',
 'Luke Kornet',
 'Denzel Valentine',
 'Daniel Gafford',
 'Shaquille Harrison',
 'Andre Drummond',
 'Tony Snell',
 'Bruce Brown',
 'Markieff Morris',
 'Reggie Jackson',
 'Luke Kennard',
 'Derrick Rose',
 'Langston Galloway',
 'Thon Maker',
 'Christian Wood',
 'Tim Frazier',
 'Sviatoslav Mykhailiuk',
 'Khyri Thomas',
 'Kevin Love',
 'Tristan Thompson',
 'Darius G

In [137]:
team_list = [team for i in range(len(pd.DataFrame(players).columns))]

In [195]:
value = [doc['teamA_stats']['Danny Green']['PTS'] for doc in cursor2]

In [27]:
foo = player_avg_stat('Giannis Antetokounmpo','PTS')
[foo[2]]

[27.714285714285715]